# <strong>Buffer Overflow</strong>

A buffer overflow is a vulnerability that occurs in programs that handle memory in an unsafe manner. Buffer overflow attacks are commonly associated with C, C++, and assembly languages, which allow direct memory management and pointer arithmetic. Programming languages that permit manual memory allocation are susceptible to this type of attack. Buffer overflows can also happen unintentionally due to programming errors, without any malicious intent. Before learning about this vulnerability, it is important to review the architecture of a computer to understand how memory is managed and how buffer overflows can affect program execution.

First, examine this diagram, which is called the Process Address Space:

<figure><center><img src="resources/buffer/buffer_overflow_diagram.png" style="width: 20%; height: 10%;"></img></center></figure>

These are the parts of the Process Address Space:
- The <strong>kernel code</strong> is the communication between the process and the hardware. Like the code block at the bottom of the address space, it does not change when it's running.
- The <strong>stack</strong> is memory that is <u>automatically</u> allocated while code is executing. The stack contains temporary variables that the program uses, and is often freed when the value is returned.
  - The stack is the "scratch pad" for the program.
  - Stacks and heaps share the same unallocated memory. There is no reserved amount of free storage that are specific to just the stack or the heap.
- The <strong>heap</strong> is memory that is <u>dynamically</u> allocated while code is executing. This memory is allocated when a pointer is created. When pointers are freed from memory, the heap shrinks.
  - To remember this: Think of a heap of dirt. When dirt (memory) is added, it grows up in size. Similar to how memory is allocated in the process address space.
- Your program's <strong>code</strong> lies at the bottom of this space. This part of the space doesn't get modified when the process is running.
  - Constant variables never change, so they lay within the code portion of the address space.
 
It can be confusing to understand why the stack and the heap are separate in the address space. The stack changes much quicker since different functions will clear the stack whenever it's complete, but the heap will need to keep the data for being the function's call. If you would like to read more about why these two types of memory are separate in the address space, check out <a href="https://stackoverflow.com/questions/8173353/why-is-memory-split-up-into-stack-and-heap">this Stack Overflow thread</a>.

<u>When variables on the stack or the heap grow too much without memory constraints, they will override important addresses that the program will depend on.</u> This is a buffer overflow. A buffer is the amount of space that a variable is allocated, but an overflow is when the buffer exceeds the amount of space that it was allocated.

<strong>This lab will contain four topics, and you will learn the following:</strong>

1. Introduction/Review of C
2. Breaking Unsafe C Functions
3. Fixing Unsafe C Functions
4. A Large-Scale Buffer Overflow Attack

In [1]:
# Setting up the lab.
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
from IPython.core.magic import register_line_magic
import os
# For accessing the nodes:
import subprocess
# For the stopexp command:
import re

# When true, it will not auto-save at each step.
runAllSteps = False

###### Used for saving notebooks. ######
import threading
# Threading required in case steps are progressed too quickly.
save_lock = threading.Lock()

# The save function itself.
def save_notebook():
    with save_lock:
        result = subprocess.run('su - USERNAME_GOES_HERE -c "/home/USERNAME_GOES_HERE/notebooks/resources/save.py buffer"', shell=True, capture_output=True, text=True)

# Creating a thread to save the notebook.
def trigger_save():
    save_thread = threading.Thread(target=save_notebook)
    save_thread.start()

###### Used for loading notebooks. ######
import queue

load_lock = threading.Lock()
result_queue = queue.Queue()

def load_notebook():
    with load_lock:
        result = subprocess.run('su - USERNAME_GOES_HERE -c "/home/USERNAME_GOES_HERE/notebooks/resources/load.py buffer"', shell=True, capture_output=True, text=True)
        result_queue.put(result)  # Put the result in the queue.

# Creating a thread to load the notebook.
def trigger_load():
    load_thread = threading.Thread(target=load_notebook)
    load_thread.start()
    load_thread.join()  # Wait for the thread to complete before adding result to the queue.
    return result_queue.get()  # Get the result from the queue.

### Step 0: Begin the experiment.

Click the button to begin creating the experiment.

In [2]:
# Click the button below to start the experiment.
def startlab(button):
    # Defining the lab name.
    labname = "bufferumd"

    # Writing the information to an empty field below the button.
    with output0:
        output0.clear_output()
        
        # First, checking if the materialization exists. May have been stopped by a previous lab.
        materialPattern = "real." + labname + ".umdsec[a-z]{1,2}"

        # Listing the materializations to find if there's an existing one for this lab.
        checkMaterial = os.popen('su - USERNAME_GOES_HERE -c "mrg list materializations"').read()
        regex = re.compile(materialPattern)
        # Getting the matches:
        match = regex.search(checkMaterial)

        if match:
            display(HTML("<span style='color: orange;'>An existing materialization for this lab already exists. </span><span>You might have ran another \
            lab without stopping this one. Attaching the existing materialization.</span>"))
            subprocess.run(f'su - USERNAME_GOES_HERE -c "mrg xdc attach xdc '+ match.group(0) + '"', capture_output=True, text=True, shell=True)
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook.</span>"))
        
        else:
            display(HTML("<span>No existing materializations are found.</span>"))
        
            # Second, start the lab.
            display(HTML("<span>Starting " + labname + " lab. This will take a few minutes to process. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            startexp = subprocess.run('su - USERNAME_GOES_HERE -c "bash /share/startexp ' + labname + '"', capture_output=True, text=True, shell=True)
            output0.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(startexp.stdout)

            # Another lab is already attached to the XDC.
            if (("XDC already attached") in startexp.stdout):
                existingLab = re.search(r"real.(.*).umdsec[a-z]{1,2}", startexp.stdout).group(1)

                # Shouldn't happen.
                if (labname == existingLab):
                    display(HTML("<span style='color: red;'>Your lab was already started. </span><span>Please continue to the next step.</span>"))

                # Detaching the existing lab, then attaching the current one.
                else:
                    display(HTML("<span style='color: orange;'>Warning: You did not stop your previous experiment. </span><span>Please stop your experiments \
                    before starting a new one. Detaching the " + existingLab + " experiment.</span>"))
                    os.popen('su - USERNAME_GOES_HERE -c "mrg xdc detach xdc.USERNAME_GOES_HERE"')
                    display(HTML("<span>Attaching the current lab.</span>"))
                    os.popen('su - USERNAME_GOES_HERE -c "mrg xdc attach xdc ' + materialPattern + '"')
    
            # Third, get the lab materials onto the node.
            display(HTML("<span>Allocating lab resources onto the node. <u>Please wait a little longer...</u></span>"))
            runlab = subprocess.run('su - USERNAME_GOES_HERE -c "bash /home/runlab ' + labname + '"', capture_output=True, text=True, shell=True)
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook. Your lab will be active for one week.</span>"))

# Creating the button.
startButton = widgets.Button(description="Start Lab")

# Creating an output area.
output0 = widgets.Output()

# Run the command on click.
startButton.on_click(startlab)

# Display the output.
display(startButton, output0)

Button(description='Start Lab', style=ButtonStyle())

Output()

If you previously stopped your lab, you may restore your progress below by clicking "Load Lab". <u>You do not have to load your lab if you signed out, closed your notebook, or exited your node(s) or XDC by using ```exit```.</u>

In [3]:
# Click the button below to load your lab.
def loadlab(b):
    with output0_2:
        output0_2.clear_output()
        display(HTML("<span>Searching for an existing lab in your notebook...</span>"))

    if (os.path.exists("/home/USERNAME_GOES_HERE/notebooks/saves/USERNAME_GOES_HERE_buffer.tar.gz")):
        with output0_2:
            output0_2.clear_output()
            display(HTML("<span>Loading your lab...</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            result = trigger_load()
            if (result.returncode == 0):
                output0_2.clear_output()
                display(HTML("<span style='color: green;'>Your lab has been successfully loaded. Please click on the <img width='20px' height='20px' style='margin-left: 1px;' src='resources/fast_forward.png'> icon at the top of your notebook to reflect your changes.</span>"))
            elif (result.returncode == 2):
                output0_2.clear_output()
                display(HTML("<span style='color: red;'>The buffer node is inaccessible. Please start your lab. If you have already started it, wait a minute and try again.</span>"))
            else:
                output0_2.clear_output()
                display(HTML("<span style='color: red;'>An error occurred while loading your lab.</span>"))

# Creating the button.
loadButton = widgets.Button(description="Load Lab")

# Creating an output area.
output0_2 = widgets.Output()

# Run the command on click.
loadButton.on_click(loadlab)

# Display the output.
display(loadButton, output0_2)

Button(description='Load Lab', style=ButtonStyle())

Output()

## <strong>Topic 1: Introduction/Review of C</strong>

C is a programming language created over 50 years ago, which was designed to be very fast and efficient. Unfortunately, C can also be a learning curve for students when learning how to program, which can make programs unsafe if students are unfamiliar with unsafe functions. Despite C being an old programming language, it's still useful for modern applications, since there are many benefits that C can provide developers. C is sometimes preferred over C++ because it's quicker, more foundational, and is more portable, meaning that it doesn't contain as many features as C++ when it comes to object-oriented design.

C (and C++) is known to be a "middle-level" language, rather than a "high-level" language. High-level languages are designed to be more human readable, easier to learn, and include lots of built-in libraries to keep code clean, and to reuse what's pre-implemented into the language.

Since C is a middle-level language, this means that it sits between a high-level language and a low-level language (assembly/binary). This means that C provides a mix of being both human readable, and it allows more control of your hardware, which is why pointers are used in C and C++. High-level languages perform these tasks automatically for you, and do not let you manage low-leveled details.

C sacrifices security to provide more control over hardware and can run much faster than high-leveled languages because it doesn't focus on preliminary checks. C does not check for memory leaks, garbage collection, array index restrictions, uninitialized variables, and a few more. This is to make the language run as fast as possible.

### Step 1: Create a Basic C Program (Part 1)

C programs can easily be written in a Unix environment, but C is <strong>not</strong> an interpreted language, meaning that it does not need to be converted into machine code first in order to be executed. Languages like Python and JavaScript are interpreted, meaning that you do not need to explicitly compile them before running them. However, C is a compiler language, which means that you will need to build them before they can be ran. If you would like to learn more about the differences between interpreted and compiled languages, you may read <a href="https://www.freecodecamp.org/news/compiled-versus-interpreted-languages/">this article</a>.

The most common compiler that developers use for C is ```gcc```, which stands for GNU Compiler Collection. This compiler is already installed on the ```buffer``` node for you, which you will use for this lab.

When working with input and output of a C program, C uses the ```stdio.h``` library, which stands for "STanDard Input Output" (stdio). A basic C file looks like this:

```
#include <stdio.h>

int main() {
    // Content goes here.

    return 0;
}
```

Returning 0 indicates that a program is successful. When a program is unsuccessful, it will return 1.

Using the template above, navigate into your home directory on your ```buffer``` node. You may access this node by typing ```ssh buffer```. A directory was created and named ```topic_1/```.  Inside of ```topic_1/```, create a file called ```step_1.c```, then write a program that returns a string of your username: ```USERNAME_GOES_HERE```.

Use the ```printf()``` statement. ```printf``` stands for "print formatted", which you will read more about in Step 4. To help you get started, consider searching online for a basic "Hello, World!" example, but replace it with your username.

You will be taught how to compile this program in the next step. If you are unsure if your program works, try skipping this step and proceed to Step 2 before running the check for Step 1.

In [4]:
# Click the button below to check your work.
step1Complete = False

# Function to check the permissions.
def step_1():
    # Required to change boolean value.
    global step1Complete

    with output1:
        output1.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_1.py 1 NA', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step1Complete = True

    elif (result.returncode == 0):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'>Your C file compiles, but it does not print your username (umdsecXX). Try again.</span>"))
            step1Complete = False
    
    elif (result.returncode == 2):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'>A file named '/home/USERNAME_GOES_HERE/topic_1/step_1.c' cannot be found. Ensure that this file exists.</span>"))
            step1Complete = False

    elif (result.returncode == 3):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'>Compiling your file resulted in an error. Somewhere in your file, your C syntax appears incorrect. Check again. Did you forget a semicolon?</span>"))
            step1Complete = False

def check_step_1(b):
    step_1()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check File")

# Creating an output area.
output1 = widgets.Output()

# Run the command on click.
button.on_click(check_step_1)

# Display the output.
display(button, output1)

Button(description='Check File', style=ButtonStyle())

Output()

### Step 2: Creating a Basic C Program (Part 2)

With your C program ready to become compiled, you will use ```gcc``` to compile your program and view the output of it.

This is the syntax for compiling a C program by using ```gcc```: ```gcc -o output_name file_name.c```

This is the breakdown:
- ```gcc``` indicates that you're executing a ```gcc``` command.
- ```-o``` indicates "output". This will indicate that the next argument will be the executable that your C file will be named as.
- ```output_name``` is the name of your executable that your C file will produce.
  - This is also called a "C binary file". This converts your C program into machine-readable language. Attempting to read this file is impossible, since this is how the computer will read your C file.
  - Note that the output name does not have an file extension.
- ```file_name.c``` is the name of the C file that you wish to compile.

Using this command, compile your C program. Name your output file as ```step_2```. Recall that your C file is named ```step_1.c```. <u>Keep doing your work inside of ```topic_1/```.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> Running ```gcc``` will detect any compiler errors. These are errors that can be detected on compile time, meaning anything that the computer knows is missing, such as a missed semicolon, calling an undefined variable/function name, using the wrong number of parameters for a statement, and more.

If any compiler errors are detected, this step will not pass.

In [5]:
# Click the button below to check your work.
step2Complete = False

# Function to check the permissions.
def step_2():
    # Required to change boolean value.
    global step2Complete

    with output2:
        output2.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_1.py 2 NA', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step2Complete = True

    elif (result.returncode == 0):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>A file named 'step_2' cannot be found in your 'topic_1' directory. Try again.</span>"))
            step2Complete = False

    elif (result.returncode == 2):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>step_2 was found, but doesn't appear to be a compiled C file. Try again, and use the gcc command.</span>"))
            step2Complete = False
    
def check_step_2(b):
    step_2()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check File")

# Creating an output area.
output2 = widgets.Output()

# Run the command on click.
button.on_click(check_step_2)

# Display the output.
display(button, output2)

Button(description='Check File', style=ButtonStyle())

Output()

### Step 3: Creating a Basic C Program (Part 3)

Lastly, you may now execute your program by running ```./step_2```. When executing a file within the same location, ```./``` needs to be in front of the file that you wish to execute. If you attempt to run ```step_2```, the Unix environment assumes that you are running a command, not executing a step. However, if you are executing a file in another directory, you may type the entire pathname, such as ```/home/USERNAME_GOES_HERE/topic_1/step_2```. The ```.``` is not required to be in front of your file name.

Try running your program. This step will not pass if it produces an error. If your program prints your username and returns 0, this step will be passed.

In [6]:
# Click the button below to check your work.
step3Complete = False

# Function to check the permissions.
def step_3():
    # Required to change boolean value.
    global step3Complete

    with output3:
        output3.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_1.py 3 NA', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step3Complete = True

    elif (result.returncode == 0):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>Your C file compiles, but it does not print your username (umdsecXX). Try again.</span>"))
            step3Complete = False
    
    elif (result.returncode == 2):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>A file named '/home/USERNAME_GOES_HERE/topic_1/step_1.c' cannot be found. Ensure that this file exists.</span>"))
            step3Complete = False

    elif (result.returncode == 3):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>Compiling your file resulted in an error. Somewhere in your file, your C syntax appears incorrect. Check again. Did you forget a semicolon?</span>"))
            step3Complete = False

def check_step_3(b):
    step_3()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check File")

# Creating an output area.
output3 = widgets.Output()

# Run the command on click.
button.on_click(check_step_3)

# Display the output.
display(button, output3)

Button(description='Check File', style=ButtonStyle())

Output()

### Step 4: Variables in C

Variables are required to have their types defined in C. These are the different types of variables that C can detect.

- Integers are defined with ```int```. These are whole numbers with no decimal value.
- Floats and doubles are defined with ```float``` and ```double``` respectively in C. Floats are precise up to 7 bits, and doubles are precise up to 15 bits without losing precision.
- Characters (or chars) are defined with ```char```.

<strong>For this step</strong>, you are going to print the sum of two numbers. Create a C file inside of ```~/topic_1/``` named ```step_4.c``` and produce an output file named ```step_4``` using ```gcc```. Then, create two variables that hold integer values. You may pick whichever two numbers that you'd like, as long as they are positive numbers and between 1 through 1,000. 

When using ```printf()``` for this, you will need to use something called a <strong>specifier</strong>. Specifiers tell ```printf``` what "type" of variable that you'd like to print. Here is an example of printing a number in C:

```
printf("My variable's value is: %i", variable_name);

>> My variable's value is: 900
```

A breakdown:
- ```printf()``` indicates that you are going to print a formatted string.
- ```"%i"``` indicates that you are going to print an integer data type. Hence, the ```"%i"```.
  - ```"%s"``` prints a string, ```"%c"``` prints a char, ```"%d"``` prints a digit/decimal (an int datatype), ```"%f"``` prints a float/double. You may also choose to print a specific amount of decimal places when using a float/double, such as ```"%.4f"```, which will print up to four decimal places.
  - You will see other types of specifiers throughout the lab, as well as learning a vulnerability using specifiers.
- ```variable_name``` is the variable that you wish to print. Make sure that the datatype of your variable aligns with your indicator within ```printf```. Otherwise, you will get a compiler error. Use the sub-bullet above for assistance with how to print specific datatypes.

You only have to print the value of the sum of two random numbers between 1 through 1,000. You do not need any additional output besides the sum. 

<u>Remember</u>: You need to store these two numbers as variables first before printing their sum.

In [7]:
# Click the button below to check your work.
step4Complete = False

# Function to check the permissions.
def step_4():
    # Required to change boolean value.
    global step4Complete

    with output4:
        output4.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_1.py 4 NA', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step4Complete = True

    elif (result.returncode == 0):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>Your two numbers were not in between 1 and 1,000. The sum is outside of the range of 2 and 2,000. Try again.</span>"))
            step4Complete = False
    
    elif (result.returncode == 2):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>One (or both) of your step_4.c and step_4 files cannot be found in the topic_1/ directory. Make sure to compile your file as step_4. Try again.</span>"))
            step4Complete = False

    elif (result.returncode == 3):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>Your file did not run properly. Re-compile your file, then fix any warnings/errors.</span>"))
            step4Complete = False

    elif (result.returncode == 4):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>Your file produced extra output. It should only produce a single number. Try again.</span>"))
            step4Complete = False

def check_step_4(b):
    step_4()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output4 = widgets.Output()

# Run the command on click.
button.on_click(check_step_4)

# Display the output.
display(button, output4)

Button(description='Check Work', style=ButtonStyle())

Output()

### Step 5: Strings in C

Strings in C work differently than other languages. This will be the first datatype that you will use in this lab that can have unsafe behavior.

All strings in C end with a character called the null terminator, which is ```\0```. Additionally, strings in C are arrays of characters whose last element is the null terminator.

This is a valid string in C:

```
char my_string[] = {'H', 'e', 'l', 'l', 'o', '\0'};
printf("%s", my_string);

>> Hello
```

Clearly, this can be very tedious. Additionally, when a string is not closed with a null character, this can occur:

```
char my_string[] = {'H', 'e', 'l', 'l', 'o'};
printf("%c", my_string[10]);

>> �
```

The program will be able to read past the end of the string and print something that is either unreadable or is a random character. When printing strings like this without null terminators, accidental data leaks can occur. Printing an incorrectly terminated string will produce unexpected output, such as this:

```
char good_string[] = {'h', 'e', 'l', 'l', 'o', '\0'};
char bad_string[] = {'w', 'o', 'r', 'l', 'd'};
printf("%s\n", bad_string);

>> worldhello
```

```good_string``` and ```bad_string``` are stored next to each other in the stack. Printing ```bad_string``` will print all data up until the first null terminator that it will find. Since ```bad_string``` wasn't terminated, it will keep reading from the stack until it encounters its first null character. Clearly, this is a security vulnerability.

Fortunately, C has a shorthand (and safer) way to store strings as variables. Initiating a variable as a ```char``` array will accept a string upon assignment, which will automatically add a null terminator. Here's an example:

```
char string_var[] = "Hello, World!";
printf("%s", string_var);
```

<strong>For this task</strong>, create a C file named ```step_5.c``` inside of ```~/topic_1/```, with an output file named ```step_5```. Then, store your username as a string into a variable named ```username```.
- Do not create a list of individual characters like the unsafe example demonstrated.
- Do not use pointers (if you know them).

Print: ```Hello, umdsecXX!``` as your final output.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> Use the example from the previous step to combine a output with a variable. Use the ```"%s"``` specifier, since this is a string.

In [8]:
# Click the button below to check your work.
step5Complete = False

# Function to check the permissions.
def step_5():
    # Required to change boolean value.
    global step5Complete

    with output5:
        output5.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_1.py 5 NA', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step5Complete = True

    elif (result.returncode == 0):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>\"Hello, USERNAME_GOES_HERE!\" doesn't appear to be the output of your program. Try again.</span>"))
            step5Complete = False
    
    elif (result.returncode == 2):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>One (or both) of your step_5.c and step_5 files cannot be found in the topic_1/ directory. Make sure to compile your file as step_5. Try again.</span>"))
            step5Complete = False

    elif (result.returncode == 3):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>Your file did not run properly. Re-compile your file, then fix any warnings/errors.</span>"))
            step5Complete = False

    elif (result.returncode == 4):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>Please use \"USERNAME_GOES_HERE\" as your username.</span>"))
            step5Complete = False

def check_step_5(b):
    step_5()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output5 = widgets.Output()

# Run the command on click.
button.on_click(check_step_5)

# Display the output.
display(button, output5)

Button(description='Check Work', style=ButtonStyle())

Output()

### Step 6: Pointers in C (Part 1)

One of the features with C being a middle-leveled language is the use of pointers. Pointers allow you to allocate memory to the heap of the Process Address Space. High-leveled languages do not use pointers, since the compiler will handle heap memory automatically. 

Whenever a variable is passed to a function's parameter in C, it's only passed a <strong>copy</strong> of the variable. This means that the value of the variable is passed to the function, but the address of the variable <strong>does not</strong> get passed.

Observe this function carefully in C:

```
#include <stdio.h>

void sum(int a, int b, int c) {
    c = a + b;
}

int main() {
    int a, b, c;
    a = 1;
    b = 2;
    c = 0;

    sum(a, b, c);

    printf("%d", c);
    return 0;
}
```

In the field below, what is the output of this function?

In [9]:
# Click the button below to check your work.
step6Complete = False

# Function to check if the student's answer was correct.
def step_6():
    # Required to change boolean values.
    global step6Complete

    # Loading, in case the check is slow.
    with output6:
        output6.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput6.value == ""):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step6Complete = False

    # Next, check to make sure that it's only a number.
    if (not str(userInput6.value).isnumeric()):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>Your answer must be a number.</span>"))
            step6Complete = False

    else:
        # Construct the SSH command for sending the student's answer to the Python file.
        check_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer "/home/.checker/section_1.py 6 '{userInput6.value}'" """
        result = subprocess.run(check_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer "echo '{userInput6.value}' > /home/.checker/responses/step_6_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: green;'>Correct!</span>"))
                step6Complete = True
        
        elif (result.returncode == 0):
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: red;'>The output that you entered is incorrect.</span>"))
                step6Complete = False

        
def check_step_6(b):
    step_6()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading6 = widgets.Output()
display(loading6)
with loading6:
    loading6.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput6 = widgets.Text(
    placeholder='Type the result of the function here',
    description='Output:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer "cat /home/.checker/responses/step_6_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Trim the newline.
userInput6.value = result.stdout[:-1]

# After the student's response was loaded, clear the output.
loading6.clear_output()

# Creating the button.
button = widgets.Button(description="Check Answer")

# Creating an output area.
output6 = widgets.Output()

# Run the command on click.
button.on_click(check_step_6)

# Display the output.
display(userInput6, button, output6)

Output()

Text(value='', description='Output:', layout=Layout(width='90%'), placeholder='Type the result of the function…

Button(description='Check Answer', style=ButtonStyle())

Output()

### Step 7: Pointers in C (Part 2)

In order to update this function so that it properly works, pointers must be introduced. A pointer is an address in memory that holds the value of a variable. Here is an example of a pointer. 

```
int* ptr;
int a = 10;
ptr = &a;
printf("Pointer: %p\n", &ptr);
printf("Value: %d", *ptr);

>> Pointer: 0x7ffdb02fe240
>> Value: 10
```

Here's a breakdown of how this works.
- Create an integer pointer named ```ptr```. This will hold the address of a variable.
- Create an integer named ```a```, which has the value 10.
- The pointer holds the <strong>address</strong> of the value. Therefore, retrieve the address of ```a``` by calling ```&a```. The ```&``` is the <u>address operator</u> in C.
- Print the <strong>address</strong> of the pointer itself by printing ```&ptr```.
  - To get the address of the variable that it holds, you would need to call ```(void*)ptr```. This provides the address that ```a``` was stored on.
- Print the <strong>value</strong> that ```ptr``` holds by printing ```*ptr```.

Your task is to recreate the function in Step 6, but use pointers so that ```a```, ```b```, and ```c``` are properly updated in ```main()```. Originally, copies of the variables were passed to ```sum()```. Now, in order for the variable ```c``` to update, you will need to use a pointer to change its value in memory. 

A template of this function is provided to you below.

```
#include <stdio.h>

void sum(int* a, int* b, int* c) {
    // Your answer here.
}

int main() {
    int a, b, c;

    a = 1;
    b = 2;
    c = 0;

    // Your answer here.

    printf("%d\n", c);
    return 0;
}
```

Upon completing the previous step, a file named ```~/topic_1/step_7.c``` was created. Click the button below to check your work.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span> 
- It's important to think about how functions and pointers work in C. ```sum()``` is a function that accepts pointers for parameters. This means that when you call ```a```, ```b```, and ```c```, they are pointers. When accessing the value of a pointer, you must have an asterisk in the variable name when using it.
- Since ```sum()``` accepts pointers, you must pass the addresses of the variables to the parameters.

<span style="color: orange"><strong><img src="resources/alert.png" style="width: 12px"> Notice:</strong></span> Starting with this step, the notebook will automatically generate the ```step_X.c``` files for you for the rest of the lab. You MUST complete the previous step before the file is generated for you. However, you are still required to compile the C file with ```gcc``` at each step. This is required anytime you change a file that's written in C.

In [10]:
# Click the button below to check your work.
step7Complete = False

# Function to check the permissions.
def step_7():
    # Required to change boolean value.
    global step7Complete

    with output7:
        output7.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_1.py 7 NA', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step7Complete = True

    elif (result.returncode == 0):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>The variable, c, is not equal to 3. Try again.</span>"))
            step7Complete = False
    
    elif (result.returncode == 2):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>One (or both) of your step_7.c and step_7 files cannot be found in the topic_1/ directory. Make sure to compile your file as step_7. Try again.</span>"))
            step7Complete = False

    elif (result.returncode == 3):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>Your file did not run properly. Re-compile your file, then fix any warnings/errors.</span>"))
            step7Complete = False

    elif (result.returncode == 4):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>The two lines that you're required to fill are not properly written. Try again.</span>"))
            step7Complete = False

def check_step_7(b):
    step_7()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output7 = widgets.Output()

# Run the command on click.
button.on_click(check_step_7)

# Display the output.
display(button, output7)

Button(description='Check Work', style=ButtonStyle())

Output()

### Step 8: Pointers in C (Part 3)

Arrays in C must be declared with a fixed length, unlike other languages which allow you to dynamically add elements to arrays. An array of char is typically used to represent a string in C, and arrays of other types (like int, float, etc.) follow similar syntax. Here's an array of integers in C:

```
int[] array_of_ints = {1, 2, 3, 4, 5};
```

If you don't want to create an array with initial values, you must declare the array with a fixed size:

```
int[5] array_of_ints.
```

When arrays in C are passed to a function, they are automatically passed by reference. This means that the function receives a pointer to the first element of the array, not a copy of the entire array. Therefore, an array does not need to be passed as a pointer explicitly to a function if you need to change any array elements within that function.

```
#include <stdio.h>

void modify_array(int arr[], int size) {
    for (int i = 0; i < size; ++i) {
        arr[i] += 1;
    }
}

int main() {
    int array_of_ints[5] = {1, 2, 3, 4, 5};
    modify_array(array_of_ints, 5);
    for (int i = 0; i < 5; ++i) {
        printf("%d ", array_of_ints[i]); // Prints 2 3 4 5 6
    }
    return 0;
}
```

Arrays are being mentioned in this topic about pointers because the size of arrays may not be known during the time of execution. This brings up the keyword ```malloc```. The word ```malloc``` is short for "memory allocation", and is used to reserve a specific amount of storage in the heap. 

This is the signature for ```malloc```: ```void *malloc(size_t size);```
- ```*malloc``` returns a pointer of whatever type that you initiate with ```*```. If it's unsuccessful, it returns an empty pointer. In this example, it's void, so it returns no type.
- ```size_t``` is an unsigned integer type. "Unsigned" means non-negative.
- ```size``` is the amount of memory that you wish to allocate in bytes in the heap.

<u>C requires that you ```free``` the variable once you finished using it.</u> This is only when you use ```malloc```. The pointer example in the previous step did not require ```free``` because it was not dynamically allocated.

Here is an example of using ```malloc``` to create an array of integers.

```
#include <stdio.h>
#include <stdlib.h> // Required for using malloc().

int main() {
    int length = 10;
    int array_of_ints = (int*)malloc(length * sizeof(int));

    // Do stuff with array_of_ints.

    free(array_of_ints);
    return 0;
}
```

For this task, you are going to be given this template:

```
#include <stdio.h>
#include <stdlib.h>

int main() {
    // Create the variables.
    int num_elements;
    const float PI = 3.14159;
    
    // Ask for user input.
    printf("Enter the number of elements: ");
    scanf("%d", &num_elements);

    // TASK 1: Use malloc to create array_of_floats.
    // It should be a pointer of type "float"!

    // Populating the array.
    for (int i = 0; i < num_elements; ++i) {
        array_of_floats[i] = i * PI;
    }

    // Printing the elements.
    printf("Array elements: ");
    for (int i = 0; i < num_elements; ++i) {
        printf("%.5f ", array_of_floats[i]);
    }
    printf("\n");

    // TASK 2: Free the array.

    return 0;
}
```

Here's a breakdown of the code:
- Create an integer variable that holds the number of elements in a list, as well as a constant named "pi".
- The program will ask for a number from you. The ```scanf()``` function takes input from the user, then assigns it to the address of ```num_elements```.
- <strong>Task 1:</strong> Create an array named ```array_of_floats``` by using malloc. Use the number of elements as the length of the array that you're allocating. As the name implies, it should be a float datatype.
- Multiply each index of the array by pi.
- Print array in a "for" loop, with up to 5 decimal precision.
- <strong>Task 2:</strong> Free ```array_of_floats``` from memory.

```step_8.c``` was automatically created for you upon completing Step 7. You are still required to use ```gcc``` in order to compile ```step_8.c```.

In [11]:
# Click the button below to check your work.
step8Complete = False

# Function to check the permissions.
def step_8():
    # Required to change boolean value.
    global step8Complete

    with output8:
        output8.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_1.py 8 NA', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step8Complete = True

    elif (result.returncode == 0):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>Running your program with '5' as input did not produce the correct results. Check again.</span>"))
            step8Complete = False
    
    elif (result.returncode == 2):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>One (or both) of your step_8.c and step_8 files cannot be found in the topic_1/ directory. Make sure to compile your file as step_8. Try again.</span>"))
            step8Complete = False

    elif (result.returncode == 3):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>Your file did not run properly. Re-compile your file, then fix any warnings/errors.</span>"))
            step8Complete = False

    elif (result.returncode == 4):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>Your call to malloc() or free() isn't correct. Try again</span>"))
            step8Complete = False

def check_step_8(b):
    step_8()

    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output8 = widgets.Output()

# Run the command on click.
button.on_click(check_step_8)

# Display the output.
display(button, output8)

Button(description='Check Work', style=ButtonStyle())

Output()

## <strong>Topic 2: Breaking Unsafe C Functions</strong>

Many functions in C are "unbounded", meaning that they do not check bounds when they are executed. This is what leads to buffer overflows. These unbounded functions are located in the ```string.h``` header file, which you may read more about <a href="https://www.geeksforgeeks.org/c-library-string-h/">here</a>.

This is a list of unsafe C functions that could lead to a buffer overflow: ```strcpy```, ```strcmp```, ```strcat```, ```strchr```, ```strspn```, ```strcspn```, ```strpbrk```, ```sttchr```, ```strstr```, ```strtok```, and ```strlen```.

In this topic, you will be breaking some of these string functions, then learn how to fix them. <u>The functions that you will be breaking are purposefully written inside of a user-defined function.</u> User-defined functions create a new "stack frame" within the Process Address Space. All variables and return addresses for this function are located in the stack.

A <strong>stack canary</strong> is a value that gets placed in the stack, which lies after the variables, but before the return address of the function. If a buffer overflow overwrites the stack canary, the program will detect this alteration and typically terminate the process with an error, such as ```stack smashing detected```. Stack canaries are not located within ```main()```, which makes it more difficult to detect a buffer overflow, unless a segmentation fault occurs or a variable becomes overwritten.

### Step 9: Breaking ```strcpy```

The ```strcpy``` function stands for "string copy". This is the signature of the ```strcpy``` command:

```char* strcpy(char* destination, const char* source);```

Parameters:
- ```destination``` is a pointer to the destination array where the content is to be copied.
- ```source``` is the C string to be copied.

<a href="https://cplusplus.com/reference/cstring/strcpy/">Link to official documentation.</a>

Upon completing Step 8, a file named ```step_9.c``` was automatically created and compiled for you. You can find this file inside of ```~/topic_2```. This is the source code:

```
#include <stdio.h>
#include <string.h>

void copy_string() {
    char *str1 = "Hello!\n";
    char str2[10];

    strcpy(str2, str1);
    
    printf("%s", str2);
}

int main() {
    copy_string();
    return 0;
}
```

Run this file and observe its functionality. Your task is to produce a buffer overflow with this function. Make changes to this function, then click "Check Work" to see if your program crashes. If your program crashes, then you will pass this step.

In [12]:
# Click the button below to check your work.
step9Complete = False

# Function to check the permissions.
def step_9():
    # Required to change boolean value.
    global step9Complete

    with output9:
        output9.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_2.py 9', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: green;'>Success! You crashed the C program.</span>"))
            step9Complete = True

    # Note: This function doesn't return zero unless it's Step 10.
    
    elif (result.returncode == 2):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>One (or both) of your step_9.c and step_9 files cannot be found in the topic_2/ directory. Make sure to compile your file as step_9. Try again.</span>"))
            step9Complete = False

    elif (result.returncode == 3):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>step_9.c has an error and doesn't compile. Check your work.</span>"))
            step9Complete = False

    elif (result.returncode == 4):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>step_9.c appears to have its function changed. Please only change the payload.</span>"))
            step9Complete = False

def check_step_9(b):
    step_9()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Payload")

# Creating an output area.
output9 = widgets.Output()

# Run the command on click.
button.on_click(check_step_9)

# Display the output.
display(button, output9)

Button(description='Check Payload', style=ButtonStyle())

Output()

### Step 10: Breaking ```strcmp```

The ```strcmp``` function stands for "string compare". This is the signature of the ```strcmp``` command:

```int strcmp (const char* str1, const char* str2);```

Parameters:
- ```str1``` and ```str2``` are strings to be compared with each other.

<a href="https://cplusplus.com/reference/cstring/strcmp/">Link to official documentation.</a>

Upon completing Step 9, a file named ```step_10.c``` was automatically created and compiled for you. You can find this file inside of ```~/topic_2```. This is the source code:

```
#include <stdio.h>
#include <string.h>

void compare_string() {
    char str1[] = "Hello";
    char str2[] = "Hello";

    int result = strcmp(str1, str2);

    if (result == 0) {
        printf("The strings are the same!\n");
    } 
    
    else {
        printf("The strings are NOT the same!\n");
    }
}

int main() {
    compare_string();
    return 0;
}
```

Run this file and observe its functionality. Your task is to make ```str1``` and ```str2``` be the same string, but print ```The strings are NOT the same!```. Your two variables are required to be named ```str1``` and ```str2```. Do not rename these, or your step will not pass.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> Review Step 5 if you are stuck.

In [13]:
# Click the button below to check your work.
step10Complete = False

# Function to check the permissions.
def step_10():
    # Required to change boolean value.
    global step10Complete

    with output10:
        output10.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_2.py 10', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: green;'>Success! You crashed the C program.</span>"))
            step10Complete = True

    elif (result.returncode == 0):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>Either your two strings are NOT equal, or you don't have two strings in your step_10.c file.</span>"))
            step10Complete = False
    
    elif (result.returncode == 2):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>One (or both) of your step_10.c and step_10 files cannot be found in the topic_2/ directory. Make sure to compile your file as step_10. Try again.</span>"))
            step10Complete = False

    elif (result.returncode == 3):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>step_10.c has an error and doesn't compile. Check your work.</span>"))
            step10Complete = False

    elif (result.returncode == 4):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>step_10.c appears to have its function changed. Please only change the payload.</span>"))
            step10Complete = False

def check_step_10(b):
    step_10()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Payload")

# Creating an output area.
output10 = widgets.Output()

# Run the command on click.
button.on_click(check_step_10)

# Display the output.
display(button, output10)

Button(description='Check Payload', style=ButtonStyle())

Output()

### Step 11: Breaking ```strcat```

The ```strcat``` function stands for "string concatenation". This is the signature of the ```strcat``` command:

```char* strcat(char* destination, const char* source);```

Parameters:
- ```destination``` is a pointer to the destination array, which should contain a C string, and be large enough to contain the concatenated resulting string.
- ```source``` is the C string to be appended. This should not overlap destination.

<a href="https://cplusplus.com/reference/cstring/strcat/">Link to official documentation.</a>

Upon completing Step 10, a file named ```step_11.c``` was automatically created and compiled for you. You can find this file inside of ```~/topic_2```. This is the source code:

```
#include <stdio.h>
#include <string.h>

void concat_string() {
    char str1[20] = "Hello,";
    char *str2 = " World!\n";

    strcat(str1, str2);

    printf("%s", str1);
}

int main() {
    concat_string();
    return 0;
}
```

Run this file and observe its functionality. Your task is to produce a buffer overflow with this function. Make the changes to the function, then click "Check Work" to see if your program crashes. If your program crashes, then you will pass this step.

In [14]:
# Click the button below to check your work.
step11Complete = False

# Function to check the permissions.
def step_11():
    # Required to change boolean value.
    global step11Complete

    with output11:
        output11.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_2.py 11', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: green;'>Success! You crashed the C program.</span>"))
            step11Complete = True

    # Note: This function doesn't return zero unless it's Step 10.
    
    elif (result.returncode == 2):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>One (or both) of your step_11.c and step_11 files cannot be found in the topic_2/ directory. Make sure to compile your file as step_11. Try again.</span>"))
            step11Complete = False

    elif (result.returncode == 3):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>step_11.c has an error and doesn't compile. Check your work.</span>"))
            step11Complete = False

    elif (result.returncode == 4):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>step_11.c appears to have its function changed. Please only change the payload.</span>"))
            step11Complete = False

def check_step_11(b):
    step_11()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Payload")

# Creating an output area.
output11 = widgets.Output()

# Run the command on click.
button.on_click(check_step_11)

# Display the output.
display(button, output11)

Button(description='Check Payload', style=ButtonStyle())

Output()

### Step 12: Breaking ```sprintf```

The ```sprintf``` function is a function that wasn't mentioned at the beginning of Topic 2, but can still be easily broken. You have already seen the ```printf``` statement many times already throughout this lab. The ```sprintf``` function means "<u>s</u>tring <u>print</u> <u>f</u>ormatted". <u>This function is not in ```string.h```, unlike the previous three functions.</u>

<strong>To put simply</strong>, this function will assign a string to a variable name that you already initiated.

This is the signature:

```int sprintf (char* str, const char* format, ...);```

Parameters: 
- ```str``` is a pointer to a buffer where the resulting C-string is stored. The buffer should be large enough to contain the resulting string.
- ```format``` is the C string that contains a format string that follows the same specifications as format in ```printf```.
- ```...``` are additional parameters that can be used, but would be unnecessary for this step.

<a href="https://cplusplus.com/reference/cstdio/sprintf/?kw=sprintf">Link to official documentation.</a>

Upon completing Step 11, a file named ```step_12.c``` was automatically created and compiled for you. You can find this file inside of ```~/topic_2```. This is the source code:

```
#include <stdio.h>

void sprintf_example() {
    char buffer[15];
    sprintf(buffer, "Hello, world!");
    printf("%s\n", buffer);
}

int main() {
    sprintf_example();
    return 0;
}

```

Run this file and observe its functionality. Your task is to continue calling ```sprintf```, but create a buffer overflow.

In [15]:
# Click the button below to check your work.
step12Complete = False

# Function to check the permissions.
def step_12():
    # Required to change boolean value.
    global step12Complete

    with output12:
        output12.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_2.py 12', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: green;'>Success! You crashed the C program.</span>"))
            step12Complete = True

    # Note: This function doesn't return zero unless it's Step 10.
    
    elif (result.returncode == 2):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'>One (or both) of your step_12.c and step_12 files cannot be found in the topic_2/ directory. Make sure to compile your file as step_12. Try again.</span>"))
            step12Complete = False

    elif (result.returncode == 3):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'>step_12.c has an error and doesn't compile. Check your work.</span>"))
            step12Complete = False

    elif (result.returncode == 4):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'>step_12.c appears to have its function changed. Please only change the payload.</span>"))
            step12Complete = False

    elif (result.returncode == 5):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'>Your step was completed, but your step_9.c file wasn't found. This file is required to be made before you can continue.</span>"))
            step12Complete = False

def check_step_12(b):
    step_12()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Payload")

# Creating an output area.
output12 = widgets.Output()

# Run the command on click.
button.on_click(check_step_12)

# Display the output.
display(button, output12)

Button(description='Check Payload', style=ButtonStyle())

Output()

## <strong>Topic 3: Fixing Unsafe C Functions</strong>

In the previous topic, you have experimented with four of the many unsafe C functions. There should be a common pattern that occurs with all of these functions. The functions broke due because of a null terminator not existing, or the bounds of a buffer (your string) were not properly constrainted, which created a "stack buffer overflow".

Now, you are going to take the four functions that you broke, and you will be applying fixes to them.

### Step 13: Fixing ```strcpy```

Your file called ```step_9.c``` had its ```strcpy``` function broken. To keep the functionality of this file, but to make it safer, you will use ```strncpy```. Here is the signature for the function:

```char* strncpy (char* destination, const char* source, size_t num);```

Parameters:
- ```destination``` is a pointer to the destination array where the content is to be copied.
- ```source``` is the C string to be copied.
- ```num``` is the maximum number of characters to be copied from source. Recall that ```size_t``` is an unsigned integer, meaning that it can't be negative. 

<a href="https://cplusplus.com/reference/cstring/strncpy/">Link to official documentation.</a>

<strong>Now, ```topic_3/``` was created in your home directory.</strong> Upon completing Step 12, a file named ```step_13.c``` was created inside of ```topic_3/```. This file is your copy of ```step_9.c``` with its vulnerability.

<u>Do not change anything except the line containing ```strcpy```.</u>

Click "Check Work" to test your fix.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> ```strncpy``` does not automatically append a null-termination string. You would have to manually add a ```\0``` at the end of the destination string. However, the main goal for this step is to just prevent a ```Segmentation Fault```. If your string does not print the first 10 characters correctly, that is fine. You are just preventing a crash from occurring for this step.

In [16]:
# Click the button below to check your work.
step13Complete = False

# Function to check the permissions.
def step_13():
    # Required to change boolean value.
    global step13Complete

    with output13:
        output13.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_3.py 13', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: green;'>Success! Your previous payload doesn't crash the file anymore.</span>"))
            step13Complete = True

    elif (result.returncode == 0):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: red;'>Your file doesn't appear to be patched. Try again.</span>"))
            step13Complete = False
    
    elif (result.returncode == 2):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: red;'>You are not using the required function(s) in step_13.c or step_9.c. If Step 9 passes successfully, then make sure that step_13.c is using strncpy.</span>"))
            step13Complete = False

    elif (result.returncode == 3):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: red;'>step_13.c doesn't compile correctly. Please fix any errors and try again.</span>"))
            step13Complete = False

    elif (result.returncode == 4):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: red;'>step_13.c and/or step_9.c is missing.</span>"))
            step13Complete = False

    elif (result.returncode == 5):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: red;'>Your exploit for Step 9 doesn't appear to work anymore. Please go back and re-crash the program.</span>"))
            step13Complete = False

def check_step_13(b):
    step_13()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Payload")

# Creating an output area.
output13 = widgets.Output()

# Run the command on click.
button.on_click(check_step_13)

# Display the output.
display(button, output13)

Button(description='Check Payload', style=ButtonStyle())

Output()

### Step 14: Fixing ```strcmp```

Your file called ```step_10.c``` had its ```strcmp``` function broken. To keep the functionality of this file, but to make it safer, you will use ```strncmp```. Here is the signature for the function:

```int strncmp (const char* str1, const char* str2, size_t num);```

Parameters:
- ```str1``` and ```str2``` are strings to be compared with each other.
- ```num``` is the maximum number of characters to be copied from source. Recall that ```size_t``` is an unsigned integer, meaning that it can't be negative. 

<a href="https://cplusplus.com/reference/cstring/strncmp/">Link to official documentation.</a>

<u>Do not change anything except the line containing ```strcmp```.</u>

Click "Check Work" to test your fix.

In [17]:
# Click the button below to check your work.
step14Complete = False

# Function to check the permissions.
def step_14():
    # Required to change boolean value.
    global step14Complete

    with output14:
        output14.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_3.py 14', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output14.clear_output()
        with output14:
            display(HTML("<span style='color: green;'>Success! Your previous payload doesn't crash the file anymore.</span>"))
            step14Complete = True

    elif (result.returncode == 0):
        output14.clear_output()
        with output14:
            display(HTML("<span style='color: red;'>Your file doesn't appear to be patched. Try again.</span>"))
            step14Complete = False
    
    elif (result.returncode == 2):
        output14.clear_output()
        with output14:
            display(HTML("<span style='color: red;'>You are not using the required function(s) in step_14.c or step_10.c. If Step 10 passes successfully, then make sure that step_14.c is using strncmp.</span>"))
            step14Complete = False

    elif (result.returncode == 3):
        output14.clear_output()
        with output14:
            display(HTML("<span style='color: red;'>step_14.c doesn't compile correctly. Please fix any errors and try again.</span>"))
            step14Complete = False

    elif (result.returncode == 4):
        output14.clear_output()
        with output14:
            display(HTML("<span style='color: red;'>step_14.c and/or step_10.c is missing.</span>"))
            step14Complete = False

    elif (result.returncode == 5):
        output14.clear_output()
        with output14:
            display(HTML("<span style='color: red;'>Your exploit for Step 10 doesn't appear to work anymore. Please go back and re-crash the program.</span>"))
            step14Complete = False

def check_step_14(b):
    step_14()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    
# Creating the button.
button = widgets.Button(description="Check Payload")

# Creating an output area.
output14 = widgets.Output()

# Run the command on click.
button.on_click(check_step_14)

# Display the output.
display(button, output14)

Button(description='Check Payload', style=ButtonStyle())

Output()

### Step 15: Fixing ```strcat```

Your file called ```step_11.c``` had its ```strcat``` function broken. To keep the functionality of this file, but to make it safer, you will use ```strcat```. Here is the signature for the function:

```char* strncat (char* destination, const char* source, size_t num);```

Parameters:
- ```destination``` is a pointer to the destination array, which should contain a C string, and be large enough to contain the concatenated resulting string.
- ```source``` is the C string to be appended. This should not overlap destination.
- ```num``` is the maximum number of characters to be copied from source. Recall that ```size_t``` is an unsigned integer, meaning that it can't be negative. 

<a href="https://cplusplus.com/reference/cstring/strncat/">Link to official documentation.</a>

<u>Do not change anything except the line containing ```strcat```.</u>

Click "Check Work" to test your fix.

In [18]:
# Click the button below to check your work.
step15Complete = False

# Function to check the permissions.
def step_15():
    # Required to change boolean value.
    global step15Complete

    with output15:
        output15.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_3.py 15', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: green;'>Success! Your previous payload doesn't crash the file anymore.</span>"))
            step15Complete = True

    elif (result.returncode == 0):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: red;'>Your file doesn't appear to be patched. Try again.</span>"))
            step15Complete = False
    
    elif (result.returncode == 2):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: red;'>You are not using the required function(s) in step_15.c or step_11.c. If Step 11 passes successfully, then make sure that step_15.c is using strncat.</span>"))
            step15Complete = False

    elif (result.returncode == 3):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: red;'>step_15.c doesn't compile correctly. Please fix any errors and try again.</span>"))
            step15Complete = False

    elif (result.returncode == 4):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: red;'>step_15.c and/or step_11.c is missing.</span>"))
            step15Complete = False

    elif (result.returncode == 5):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: red;'>Your exploit for Step 11 doesn't appear to work anymore. Please go back and re-crash the program.</span>"))
            step15Complete = False

def check_step_15(b):
    step_15()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Payload")

# Creating an output area.
output15 = widgets.Output()

# Run the command on click.
button.on_click(check_step_15)

# Display the output.
display(button, output15)

Button(description='Check Payload', style=ButtonStyle())

Output()

### Step 16: Fixing ```sprintf```

Your file called ```step_12.c``` had its ```sprintf``` function broken. To keep the functionality of this file, but to make it safer, you will use ```snprintf```. Here is the signature for the function:

```int snprintf (char* s, size_t n, const char* format, ...);```

Parameters: 
- ```str``` is a pointer to a buffer where the resulting C-string is stored. The buffer should be large enough to contain the resulting string.
- ```n``` is the maximum number of characters to be copied from source. Recall that ```size_t``` is an unsigned integer, meaning that it can't be negative. 
- ```format``` is the C string that contains a format string that follows the same specifications as format in ```printf```.
- ```...``` are additional parameters that can be used, but would be unnecessary for this step.

<a href="https://cplusplus.com/reference/cstdio/snprintf/?kw=snprintf">Link to official documentation.</a>

<u>Do not change anything except the line containing ```sprintf```.</u>

Click "Check Work" to test your fix.

In [19]:
# Click the button below to check your work.
step16Complete = False

# Function to check the permissions.
def step_16():
    # Required to change boolean value.
    global step16Complete

    with output16:
        output16.clear_output()
        display(HTML("<span>Please wait. This step will take longer if you passed. Your notebook is setting up Topic 4...<img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_3.py 16', shell=True, capture_output=True, text=True)
    
    if (result.returncode == 1):
        output16.clear_output()
        with output16:
            display(HTML("<span style='color: green;'>Success! Your previous payload doesn't crash the file anymore.<br></span>"))
            display(HTML("<span style='color: green;'>Please wait. Your next topic is being set up...<img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

            # Check if the topic_4 directory is already made.
            result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer test -d "/home/USERNAME_GOES_HERE/topic_4"', shell=True)
            if (result.returncode == 1):
                # Run topic 4 setup.
                subprocess.run("ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/create_topic_4.sh", shell=True)
                
            # Setup is complete. Remove the loading message.
            output16.clear_output()
            with output16:
                display(HTML("<span style='color: green;'>Success! Your previous payload doesn't crash the file anymore.<br></span>"))
                step16Complete = True

    elif (result.returncode == 0):
        output16.clear_output()
        with output16:
            display(HTML("<span style='color: red;'>Your file doesn't appear to be patched. Try again.</span>"))
            step16Complete = False
    
    elif (result.returncode == 2):
        output16.clear_output()
        with output16:
            display(HTML("<span style='color: red;'>You are not using the required function(s) in step_16.c or step_12.c. If Step 12 passes successfully, then make sure that step_16.c is using strncat.</span>"))
            step16Complete = False

    elif (result.returncode == 3):
        output16.clear_output()
        with output16:
            display(HTML("<span style='color: red;'>step_16.c doesn't compile correctly. Please fix any errors and try again.</span>"))
            step16Complete = False

    elif (result.returncode == 4):
        output16.clear_output()
        with output16:
            display(HTML("<span style='color: red;'>step_16.c and/or step_12.c is missing.</span>"))
            step16Complete = False

    elif (result.returncode == 5):
        output16.clear_output()
        with output16:
            display(HTML("<span style='color: red;'>Your exploit for Step 12 doesn't appear to work anymore. Please go back and re-crash the program.</span>"))
            step16Complete = False

def check_step_16(b):
    step_16()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Payload")

# Creating an output area.
output16 = widgets.Output()

# Run the command on click.
button.on_click(check_step_16)

# Display the output.
display(button, output16)

Button(description='Check Payload', style=ButtonStyle())

Output()

## <strong>Topic 4: A Large-Scale Buffer Overflow Attack</strong>

In this final topic, you will be accessing a nuclear reactor simulator with four different vulnerabilities which can commonly appear in programs written in C:

1. <strong>Buffer Overflow:</strong> When more bytes are put into a buffer than the buffer has space for.
2. <strong>Off-By-One Errors:</strong> When a counting or inequality issue results in a arithmetic mistake of 1.
3. <strong>Integer Overflow/Underflow and Sign Errors:</strong> When the "wrapping" of an integer type, or an incorrect sign causes something unexpected to happen.
4. <strong>String Format Vulnerabilities:</strong> When a user is allowed to provide a format string to a ```printf``` statement.

You have already explored plenty of buffer overflow examples, as well as string format vulnerabilities. Off-By-One errors and integer errors are common mistakes that can occur in programming. However, as said before, C does not perform preliminary checks. This means when a small arithmetic error occurs, it may not be detected, and will cause logic errors.

<u>The scenario</u>: Wormwood is a nuclear reactor simulator. Written by an inexperienced C developer, plenty of mistakes are littered throughout the simulator. Many potential issues are present, and all it takes is for someone (either experienced or unexperienced) to use the wrong input, and catastrophe could occur.

You have already practice a lot of buffer overflow. These are the other three vulnerabilities that are present in the simulator:

<strong>Off-By-One errors</strong> occur when the arithmetic of a "for" loop is mishandled. Here's a common off-by-one error:

```
char list[] = {1, 2, 3, 4};
for (int i = 0; i <= 4; ++i) {
    printf("%d ", list[i]);
}
```

Which prints: ```1 2 3 4 0```

This does NOT cause an error. Despite accessing more elements than the list contains. An off-by-one error doesn't occur in just for loops. This can occur anytime that you're trying to access a specific element from an array or a char from a string. The error is most common when indices are not double-checked, since novice programmers tend to forget that indices start at 0, not 1.

<strong>Integer overflow/underflow and sign errors</strong> occur when a number "rolls over" the maximum number of the integer's type. Take a look at this code:

```
#include <limits.h>
#include <stdio.h>

int main() {
    printf("%d", INT_MAX);
    return 0;
}

>> 2147483647
```

The ```INT_MAX``` constant is the highest number that an integer can be. Attempting to add one to ```INT_MAX``` will result in this:

```
#include <limits.h>
#include <stdio.h>

int main() {
    printf("%d", INT_MAX + 1);
    return 0;
}

>> main.c: In function ‘main’:
main.c:5:26: warning: integer overflow in expression of type ‘int’ results in ‘-2147483648’ [-Woverflow]
    5 |     printf("%d", INT_MAX + 1);
      |                          ^

>> -2147483648
```

The integer has rolled over, and is now ```INT_MIN```. This is similar to rolling over 999,999 miles on an odometer, where the odometer will read 000,000 as the new mileage because it cannot output 1,000,000.

A <strong>string format vulnerability</strong> occurs when a ```printf``` statement is used, but a format (such as ```%s```, ```%d```, ```%f```, etc) isn't provided. When a user provides an input that contains a format specifier, then information from the stack may be shown. Suppose a hacker sends ```%x %x %p %p %s %s``` through user input. This will fool ```printf``` to print two hex values, two pointers, and two random elements from the stack.

If you would like to read more about this type of vulnerability, you may visit <a href="https://www.geeksforgeeks.org/format-string-vulnerability-and-prevention-with-example/">this article</a>.

### Step 17: Crashing the Control Program

Access the Wormwood simulator by navigating into ```~/topic_4/wormwood_test/```, then execute ```./run.sh```. Your first goal is to attempt to sign in as the super user. 

In the field below, type your input that allowed you to crash the program through a buffer overflow.

<span style="color: orange"><strong><img src="resources/alert.png" style="width: 12px"> Notice:</strong></span> A warning will be shown when you use ```./run.sh```. This is intentional, so you do not need to try fixing it right away.

In [20]:
# Click the button below to check your work.
step17Complete = False

# Function to check if the student's answer was correct.
def step_17():
    # Required to change boolean values.
    global step17Complete

    # Loading, in case the check is slow.
    with output17:
        output17.clear_output()
        display(HTML("<span>Please wait. Your input is being typed into Wormwood. Please wait at least 5 seconds...<img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput17.value == ""):
        output17.clear_output()
        with output17:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step17Complete = False

    # Next, run the input through.
    else:
        # Sanitizing the string.
        answer = ((userInput17.value).strip()).replace('\n', '')
        
        # Construct the SSH command for sending the student's answer to the Python file.
        # IMPORTANT: Use the -tt tag when running this SSH command. This will treat the command more like a "local" command on the node.
        # This is required, since there are some conflicts regarding directories and running ./run.sh.
        check_command = f"""ssh -tt -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_4.py 17 '{answer}' 0 """
        result = subprocess.run(check_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer "echo '{answer}' > /home/.checker/responses/step_17_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 1):
            output17.clear_output()
            with output17:
                display(HTML("<span style='color: green;'>Your payload successfully crashed the program.</span>"))
                step17Complete = True
        
        elif (result.returncode == 0):
            output17.clear_output()
            with output17:
                display(HTML("<span style='color: red;'>The output that you entered is incorrect.</span>"))
                step17Complete = False

        elif (result.returncode == 0):
            output17.clear_output()
            with output17:
                display(HTML("<span style='color: red;'>The program timed out. Your input may not have terminated. Check your payload.</span>"))
                step17Complete = False
        
def check_step_17(b):
    step_17()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading17 = widgets.Output()
display(loading17)
with loading17:
    loading17.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput17 = widgets.Text(
    placeholder='Type the payload that crashed your program.',
    description='Payload:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer "cat /home/.checker/responses/step_17_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput17.value = result.stdout

# After the student's response was loaded, clear the output.
loading17.clear_output()

# Creating the button.
button = widgets.Button(description="Check Answer")

# Creating an output area.
output17 = widgets.Output()

# Run the command on click.
button.on_click(check_step_17)

# Display the output.
display(userInput17, button, output17)

Output()

Text(value='', description='Payload:', layout=Layout(width='90%'), placeholder='Type the payload that crashed …

Button(description='Check Answer', style=ButtonStyle())

Output()

### Step 18: Accessing the ```super```'s Account

On the sign-in page, figure out what ```super```'s password is by sending input through the authentication page. You are not going to be graded on what the password is. Instead, you are going to be graded on the exploit that allowed you to view the password.

Using the blank below, type the payload that you used in order to display the super user's password onto the screen. This payload will be passed into the Wormwood lab and tested to see if it returns the password of ```super```.

In [21]:
# Click the button below to check your work.
step18Complete = False

# Function to check if the student's answer was correct.
def step_18():
    # Required to change boolean values.
    global step18Complete

    # Loading, in case the check is slow.
    with output18:
        output18.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput18.value == ""):
        output18.clear_output()
        with output18:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step18Complete = False

    # Next, run the input through.
    else:
        # Construct the SSH command for sending the student's answer to the Python file.
        check_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer \"/home/.checker/section_4.py 18 '{(userInput18.value).strip()}' 0 \""""
        result = subprocess.run(check_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer "echo '{userInput18.value}' > /home/.checker/responses/step_18_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        if (result.returncode == 1):
            output18.clear_output()
            with output18:
                display(HTML("<span style='color: green;'>Your payload successfully crashed the program.</span>"))
                step18Complete = True
        
        elif (result.returncode == 0):
            output18.clear_output()
            with output18:
                display(HTML("<span style='color: red;'>The output that you entered is incorrect.</span>"))
                step18Complete = False

        elif (result.returncode == 0):
            output18.clear_output()
            with output18:
                display(HTML("<span style='color: red;'>The program timed out. Your input may not have terminated. Check your payload.</span>"))
                step18Complete = False
        
def check_step_18(b):
    step_18()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading18 = widgets.Output()
display(loading18)
with loading18:
    loading18.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput18 = widgets.Text(
    placeholder='Type the payload that displays the password.',
    description='Payload:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer "cat /home/.checker/responses/step_18_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput18.value = result.stdout

# After the student's response was loaded, clear the output.
loading18.clear_output()

# Creating the button.
button = widgets.Button(description="Check Answer")

# Creating an output area.
output18 = widgets.Output()

# Run the command on click.
button.on_click(check_step_18)

# Display the output.
display(userInput18, button, output18)

Output()

Text(value='', description='Payload:', layout=Layout(width='90%'), placeholder='Type the payload that displays…

Button(description='Check Answer', style=ButtonStyle())

Output()

### Step 19: Off-By-One Error

After signing into ```super```'s password, you now have access to the nuclear reactor. 

Your next goal is to find the <strong>off-by-one</strong> error.

<u>For the first input field</u>, type ```super```'s password that allowed you to sign in.

<u>For the second input field</u>, type the menu option (single letter) where you suspect an off-by-one error occurs.

<u>For the third input field</u>, type the value that causes the off-by-one error.

In [22]:
# Click the button below to check your work.

# Required for formatting student's strings:
import shlex
step19Complete = False

# Function to save the short answer.
def step_19():
    # Required to change boolean values.
    global step19Complete

    # Loading, in case saving is slow.
    with output19:
        output19.clear_output()
        display(HTML("<span>Please wait. Your input is being typed into Wormwood. Please wait at least 10-30 seconds...<img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # Check for empty strings.
    if (userInput19_1.value == "" or userInput19_2.value == "" or userInput19_3.value == ""):
        output19.clear_output()
        with output19:
            display(HTML("<span style='color: red;'>You did not type a response for one of your memos.</span>"))
            step19Complete = False

    # Some more checks.
    elif (len(userInput19_2.value) != 1):
        output19.clear_output()
        with output19:
            display(HTML("<span style='color: red;'>The menu option must be a single letter.</span>"))
            step19Complete = False

    else:
        # Check to see if Step 20 was complete yet. If so, compare the answers.
        result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer "cat /home/.checker/responses/step_20_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
        other_response = result.stdout

        saved_input = shlex.quote(str(userInput19_1.value).strip() + " " + str(userInput19_2.value).strip() + " " + str(userInput19_3.value).strip())

        # Exit. This is a copied response from Step 20.
        if ('\'' + other_response[:-1] + '\'' == saved_input):
            output19.clear_output()
            with output19:
                display(HTML("<span style='color: red;'>This response is the same as Step 20. Please use a different payload.</span>"))
                step19Complete = False

        # Exit. This menu option does not have off-by-one error.
        elif (userInput19_2.value != chr(114)):
            output19.clear_output()
            with output19:
                display(HTML("<span style='color: red;'>Hint: An off-by-one error does not occur in this menu option.</span>"))
                step19Complete = False

        else:
            # Construct the SSH command for saving the student's response.
            save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer "echo '{saved_input}' > /home/.checker/responses/step_19_answer.txt" """
            save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            
            user_input = ((userInput19_1.value).strip() + " " + (userInput19_2.value).strip() + " " + (userInput19_3.value).strip())
            
            # Construct the SSH command for sending the student's answer to the Python file.
            # IMPORTANT: Use the -tt tag when running this SSH command. This will treat the command more like a "local" command on the node.
            # This is required, since there are some conflicts regarding directories and running ./run.sh.
            check_command = f"""ssh -tt -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer "/home/.checker/section_4.py 19 '{user_input}' 0" """

            result = subprocess.run(check_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            
            if (result.returncode == 1):
                output19.clear_output()
                with output19:
                    display(HTML("<span style='color: green;'>Correct! This payload causes an off-by-one error.</span>"))
                    step19Complete = True
        
            elif (result.returncode == 0):
                output19.clear_output()
                with output19:
                    display(HTML("<span style='color: red;'>This payload does not cause an off-by-one error. Try again.</span>"))
                    step19Complete = False
    
            elif (result.returncode == 2):
                output19.clear_output()
                with output19:
                    display(HTML("<span style='color: red;'>The program timed out. This means your payload failed to crash the program. Try again.</span>"))
                    step19Complete = False

def check_step_19(b):
    step_19()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading19 = widgets.Output()
display(loading19)
with loading19:
    loading19.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput19_1 = widgets.Text(
    placeholder='Type the password that you found.',
    description='Password:',
    layout=widgets.Layout(width='90%')
)

userInput19_2 = widgets.Text(
    placeholder='Type the menu option (one letter).',
    description='Option:',
    layout=widgets.Layout(width='90%')
)

userInput19_3 = widgets.Text(
    placeholder='Type the value that causes the error.',
    description='Payload:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer "cat /home/.checker/responses/step_19_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Process the output, then separate them.
if (result != ""):
    answers = (result.stdout).split(" ")

    if (len(answers) == 3):
        userInput19_1.value = answers[0]
        userInput19_2.value = (answers[1])
        userInput19_3.value = (answers[2])[:-1]

    else:
        userInput19_1.value = ""
        userInput19_2.value = ""
        userInput19_3.value = ""

# After the student's response was loaded, clear the output.
loading19.clear_output()

# Creating the feedback output area.
output19 = widgets.Output()

# Creating the button.
button = widgets.Button(description="Save Response")

# Run the command on click.
button.on_click(check_step_19)

# Display the output.
display(userInput19_1, userInput19_2, userInput19_3, button, output19)

Output()

Text(value='', description='Password:', layout=Layout(width='90%'), placeholder='Type the password that you fo…

Text(value='', description='Option:', layout=Layout(width='90%'), placeholder='Type the menu option (one lette…

Text(value='', description='Payload:', layout=Layout(width='90%'), placeholder='Type the value that causes the…

Button(description='Save Response', style=ButtonStyle())

Output()

### Step 20: Integer Overflow/Underflow

After signing into ```super```'s password, you now have access to the nuclear reactor. 

Your next goal is to find the <strong>integer overflow</strong> error.

<u>For the first input field</u>, type ```super```'s password that allowed you to sign in.

<u>For the second input field</u>, type the menu option (single letter) where you suspect an integer overflow occurs.

<u>For the third input field</u>, type the value that causes the integer overflow error.

In [23]:
# Click the button below to check your work.

# Required for formatting student's strings:
import shlex
step20Complete = False

# Function to save the short answer.
def step_20():
    # Required to change boolean values.
    global step20Complete

    # Loading, in case saving is slow.
    with output20:
        output20.clear_output()
        display(HTML("<span>Please wait. Your input is being typed into Wormwood. Please wait at least 10-30 seconds...<img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # Check for empty strings.
    if (userInput20_1.value == "" or userInput20_2.value == "" or userInput20_3.value == ""):
        output20.clear_output()
        with output20:
            display(HTML("<span style='color: red;'>You did not type a response for one of your memos.</span>"))
            step20Complete = False

    # Some more checks.
    elif (len(userInput20_2.value) != 1):
        output20.clear_output()
        with output20:
            display(HTML("<span style='color: red;'>The menu option must be a single letter.</span>"))
            step20Complete = False

    else:
        # Check to see if Step 20 was complete yet. If so, compare the answers.
        result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer "cat /home/.checker/responses/step_19_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
        other_response = result.stdout

        saved_input = shlex.quote(str(userInput20_1.value).strip() + " " + str(userInput20_2.value).strip() + " " + str(userInput20_3.value).strip())

        # Exit. This is a copied response from Step 19.
        if ('\'' + other_response[:-1] + '\'' == saved_input):
            output20.clear_output()
            with output20:
                display(HTML("<span style='color: red;'>This response is the same as Step 19. Please use a different payload.</span>"))
                step20Complete = False

        # Exit. This menu option does not have integer overflow.
        elif (userInput20_2.value != chr(102)):
            output20.clear_output()
            with output20:
                display(HTML("<span style='color: red;'>Hint: Integer overflow/underflow does not occur in this menu option.</span>"))
                step20Complete = False

        else:
            # Construct the SSH command for saving the student's response.
            save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer "echo '{saved_input}' > /home/.checker/responses/step_20_answer.txt" """
            save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            
            user_input = ((userInput20_1.value).strip() + " " + (userInput20_2.value).strip() + " " + (userInput20_3.value).strip())
            
            # Construct the SSH command for sending the student's answer to the Python file.
            # IMPORTANT: Use the -tt tag when running this SSH command. This will treat the command more like a "local" command on the node.
            # This is required, since there are some conflicts regarding directories and running ./run.sh.
            check_command = f"""ssh -tt -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer "/home/.checker/section_4.py 20 '{user_input}' 0" """
            result = subprocess.run(check_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            
            if (result.returncode == 1):
                output20.clear_output()
                with output20:
                    display(HTML("<span style='color: green;'>Correct! This payload causes an integer overflow/underflow.</span>"))
                    step20Complete = True
        
            elif (result.returncode == 0):
                output20.clear_output()
                with output20:
                    display(HTML("<span style='color: red;'>This payload does not cause an off-by-one error. Try again.</span>"))
                    step20Complete = False
    
            elif (result.returncode == 2):
                output20.clear_output()
                with output20:
                    display(HTML("<span style='color: red;'>The program timed out. This means your payload failed to crash the program. Try again.</span>"))
                    step20Complete = False

def check_step_20(b):
    step_20()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading20 = widgets.Output()
display(loading20)
with loading20:
    loading20.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput20_1 = widgets.Text(
    placeholder='Type the password that you found.',
    description='Password:',
    layout=widgets.Layout(width='90%')
)

userInput20_2 = widgets.Text(
    placeholder='Type the menu option (one letter).',
    description='Option:',
    layout=widgets.Layout(width='90%')
)

userInput20_3 = widgets.Text(
    placeholder='Type the value that causes the error.',
    description='Payload:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer "cat /home/.checker/responses/step_20_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Process the output, then separate them.
if (result != ""):
    answers = (result.stdout).split(" ")

    if (len(answers) == 3):
        userInput20_1.value = answers[0]
        userInput20_2.value = (answers[1])
        userInput20_3.value = (answers[2])[:-1]

    else:
        userInput20_1.value = ""
        userInput20_2.value = ""
        userInput20_3.value = ""

# After the student's response was loaded, clear the output.
loading20.clear_output()

# Creating the feedback output area.
output20 = widgets.Output()

# Creating the button.
button = widgets.Button(description="Save Response")

# Run the command on click.
button.on_click(check_step_20)

# Display the output.
display(userInput20_1, userInput20_2, userInput20_3, button, output20)

Output()

Text(value='', description='Password:', layout=Layout(width='90%'), placeholder='Type the password that you fo…

Text(value='', description='Option:', layout=Layout(width='90%'), placeholder='Type the menu option (one lette…

Text(value='', description='Payload:', layout=Layout(width='90%'), placeholder='Type the value that causes the…

Button(description='Save Response', style=ButtonStyle())

Output()

### Step 21: Fixing All Vulnerabilities

For the last step, you are going to fix all of the vulnerabilities that you found between steps 17-20.

<strong>Navigate into the OTHER Wormwood directory that's titled ```~/topic_4/wormwood_fix/```.</strong> The fixes to the Wormwood program must be in here so that your previous steps will still pass.

Here are some tips to help you figure out where these issues occur:
- A <strong>buffer overflow</strong> occurs because the program uses one of the four functions that you broke in Topic 2.
- When you found the <strong>string vulnerability</strong> error, you should've found that the password was displayed when the program printed the username. Find the ```console_printf``` command that does this, and change its format to what you've used in Topic 1. Keep using ```console_printf```.
- The <strong>off-by-one</strong> error can be a little trickier to find. You know which menu option has the off-by-one error. Find the function in ```wormwood.c``` that this menu option uses ```void set_XXXXX(void)```. A comment is made that checks to make sure that a value isn't too high. This will call a function inside of ```reactor.c```, which returns a boolean if a certain condition is matched. Fix the sign that causes the off-by-one error.
- The <strong>integer overflow/underflow</strong> error occurs because the bounds are not checked for your payload. Find the other function in ```wormwood.c``` which calls the menu option with the integer overflow/underflow. The function checks one boundary that your input can't reach. Make it check another boundary.

When you believe that you have fixed all bugs, click "Check Work" below. Your four inputs above will be used in this check, and if all of them do not break the C program, then you pass the final step.

In [24]:
# Click the button below to check your work.
step21Complete = False

# Function to check the permissions.
def step_21():
    # Required to change boolean value.
    global step21Complete

    with output21:
        output21.clear_output()
        display(HTML("<span><strong>Your results will be printed here momentarily.</strong> It will take roughly 40-60 seconds to finish this test. Please wait until Step 20 is complete!</span><br>"))

    correct_steps = []
    
    for i in range (17, 21):
        # Creates an additional message when testing Step 19.
        if (i == 19):
            with output21:
                display(HTML("<span><em>(These next two steps will take some additional time to pass. When the payload gets sent, the script will take 15-20 seconds for the payload to work, then see if it crashes.)</em></span><br>"))
            
        # Using the -tt command once again, because this needs to be ran at the kernel level in order for the file to work.
        # Once again, not sure why it's needed, but it will make the script function properly.
        command = 'ssh -tt -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@buffer /home/.checker/section_4_q21.py ' + str(i)
        result = subprocess.run(command, shell=True, capture_output=True, text=True)

        if (result.returncode == 4):
            with output21:
                output21.clear_output()
                display(HTML("<span>An error occurred while running this step. Please contact your professor/TA.<br><br>This test has ended.</span><br>"))
                break
        
        if (result.returncode == 1):
            with output21:
                display(HTML("<span style='color: green;'><strong>Step " + str(i) + "</strong>: Success! This vulnerability appears to be fixed.</span><br>"))
                correct_steps.append(1)

        elif (result.returncode == 0):
            with output21:
                if (i == 18):
                    display(HTML("<span style='color: red;'><strong>Step " + str(i) + "</strong>: Your string vulnerability patch did not work. Make sure that the password field prints as: \"Password for user 'super': \".</span><br>"))
                
                else:
                    display(HTML("<span style='color: red;'><strong>Step " + str(i) + "</strong>: The previous payload for this step was timed out. The program did not crash. Go back to Step " + str(i) + " and retry your payload.</span><br>"))
                correct_steps.append(0)
        
        elif (result.returncode == 2):
            with output21:
                display(HTML("<span style='color: red;'><strong>Step " + str(i) + "</strong>: An error occurred when running this check. Please contact your professor or TA.</span><br>"))
                correct_steps.append(2)

        elif (result.returncode == 3):
            with output21:
                display(HTML("<span style='color: red;'><strong>Step " + str(i) + "</strong>: Your exploit broke the original Wormwood code, but it still broke on your fixed code. Try again.</span><br>"))
                correct_steps.append(3)

    step21Complete = True
    for code in correct_steps:
        if (code != 1):
            step21Complete = False
            break

    with output21:
        if (step21Complete):
            display(HTML("<br><span style='color: green;'>Congratulations! The file is patched.</span>"))
        
        else:
            display(HTML("<br><span style='color: red;'>There are some errors in your patch. Please review the feedback above, then fix your work.</span>"))

def check_step_21(b):
    step_21()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Patch")

# Creating an output area.
output21 = widgets.Output()

# Run the command on click.
button.on_click(check_step_21)

# Display the output.
display(button, output21)

Button(description='Check Patch', style=ButtonStyle())

Output()

## <strong>Grading</strong>

To check your overall grade, click on the button below.

<strong>*Warning:* Your submission will be ran on an unmodified copy of this notebook</strong>. If you tinkered with any of the input cells to pass a step, you will not get full credit.

In [25]:
# Click the button below to check your overall grade.
steps_to_check = [step_1, step_2, step_3, step_4, step_5, step_6, step_7, step_8, step_9, step_10, step_11, step_12, step_13, step_14, step_15, step_16, step_17, step_18, step_19, step_20, step_21]   

# Function to calculate grade after refreshing the cell
def calculate_grade(b):
    # To not auto-save at each step.
    global runAllSteps
    runAllSteps = True

    with gradeOutput:
        gradeOutput.clear_output()
        display(HTML("<span>Testing all steps. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # Required for checking the boolean values.
    for func in steps_to_check:
        func()  # Call each function in order.

    # Assuming steps are updated above this cell in some way
    steps = [step1Complete, step2Complete, step3Complete, step4Complete, step5Complete, step6Complete, step7Complete, step8Complete, step9Complete, step10Complete, step11Complete, step12Complete, step13Complete, step14Complete, step15Complete, step16Complete, step17Complete, step18Complete, step19Complete, step20Complete, step21Complete]
    output = ""
    stepsCorrect = 0
    numOfSteps = len(steps)

    for i in range(numOfSteps):
        if steps[i]:
            stepsCorrect += 1
            output += "<div style='color: green;'>Step " + str(i + 1) + " is complete.</div>"
        else:
            output += "<div style='color: red;'>Step " + str(i + 1) + " is incomplete.</div>"

    output += "<div style='color: black;'>You have " + str(stepsCorrect) + " out of " + str(numOfSteps) + " steps completed.</div>"

    with gradeOutput:
        gradeOutput.clear_output()
        display(HTML(output))

    # Auto-save at the very end.
    runAllSteps = False
    if (not runAllSteps):
        trigger_save()
    

# Create a button to refresh the cell and another to calculate grade.
grade_button = widgets.Button(description="Calculate Grade")

# Link buttons to functions.
grade_button.on_click(calculate_grade)

# Output area.
gradeOutput = widgets.Output()

# Display the buttons and output.
display(grade_button, gradeOutput)

Button(description='Calculate Grade', style=ButtonStyle())

Output()

### Stopping the Lab

Once you are done with the lab, click on the "Stop Lab" button below. <strong>This will delete your materialization, which will delete all of the lab's resources.</strong> Your progress is saved automatically in ```~/notebooks/submissions```. This folder is stored on your XDC, and will not be deleted upon stopping the lab.

In [26]:
# Click the button below to stop the experiment.
def stoplab(button):
    # Check to make sure that the student wants to confirm ending the lab.
    if (confirm.value == False):
        with stop_output:
            stop_output.clear_output()
            display(HTML("<newline><span style='color: red;'>Please confirm that you wish to end the lab.</span>"))

    else:
        # Defining the lab name.
        labname = "bufferumd"
    
        # Writing the information to an empty field below the button.
        with stop_output:
            stop_output.clear_output()
            
            display(HTML("<span>Stopping the " + labname + " lab. This will take a minute to process. Please wait.</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            stopexp = subprocess.run('su - USERNAME_GOES_HERE -c "bash /share/stopexp ' + labname + 'umd"', capture_output=True, text=True, shell=True)
            stop_output.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(stopexp.stdout)
    
            display(HTML("<newline><span style='color: green;'><strong>Your lab has been ended.</strong></span>"))

# Creating the button.
stopButton = widgets.Button(description="Stop Lab")

# Create a confirmation check.
confirm = widgets.Checkbox(
    value=False,
    description='Confirm',
    disabled=False,
    indent=False
)

# Creating an output area.
stop_output = widgets.Output()

# Run the command on click.
stopButton.on_click(stoplab)

# Display the output.
display(confirm, stopButton, stop_output)

Checkbox(value=False, description='Confirm', indent=False)

Button(description='Stop Lab', style=ButtonStyle())

Output()